# 15. Die RSA-Verschlüsselung

### Inhalt:

1. [Prinzip](#prinzip)
2. [Berechnung der Schlüssel](#schluessel)
3. [Verschlüsseln einer Nachricht](#verschl)
4. [Entschlüsseln einer Nachricht](#entschl)
5. [Programm mit variabler Eingabe](#vareing)
6. [Verschlüsselung Knacken](#knacken)
7. [Die  $168$ Primzahlen von $2$ bis $1000$](#primzahlen)
8. [Eigene Experimente](#experimente)


Benannt nach Ronald **R**ivest, Adi **S**hamir und Leonard **A**dleman, geört diese Verschlüsselungstechnik zu den *Public-Key*-Verfahren,  mit einem öffentlichen und einem Privaten Schlüssel. Mit dem öffentlichen Schlüssel werden die Daten verschlüsselt und mit dem privaten Schlüssel werden sie wieder entschlüsselt. Das Verfahren basiert auf der Produktbildung von grossen Primzahlen.

Alle Nutzer können ihre Daten mit demselben Schlüssel verschlüsseln aber nicht entschlüsseln. Entschlüsseln lassen sich die Daten nur mit dem privaten Schlüssel, z. B. auf dem Server. Man spricht deshalb auch von asymmetrischer Verschlüsselung.

Das Beispiel hier ist im Buch in Kap. 20.7 genau erläutert.  Es stammt aus der ürsprünglichen Arbeit von Rivest, Shamir und Adleman, [*], Wir geben hier lediglich den Programmcode, damit Sie selbst weiterexperimentieren können. Ein anderes Beispiel finden Sie auf dieser [Webseite](https://www.cryptool.org/de/cto/rsa-step-by-step/).

<a name="prinzip"></a>
### 1. Prinzip

Man arbeitet mit einem öffentlichen Schlüssel $(e,n)$ und einem privaten Schlüssel $(d,n)$ der öffentliche Schlüssel wird zum verschlüsseln benutzt. Eine Nachricht in Zahlenform $m$ (message) wird damit folgendermassen verschlüsselt:
$$ x=m^e\ mod\ n$$
Der Kryptotext $x$ wird übermittelt und beim Empfänger mit dem privaten Schlüssel entschlüsselt, gemäss:
$$mr=x^d\ mod\ n$$
Dabei ist $mr$ die rekonstruierte Nachricht (Zahl). Wenn alles funktioniert hat, ist $mr=m$.

Die Zahl $n$ wird aus zwei möglichst grossen Primzahlen $p$ und $q$ als Produkt dieser Primzahlen  gebildet $n=p*q$. Nach welchem Verfahren die beiden Zahlen $e$ und $d$ gebildet werden, wird weiter unten erklärt.

Da $n$ auch im öffentlichen Schlüssel enthalten ist müsste man nur noch die Zahl $d$ kennen um auch den privaten Schlüssel zu haben. Die Zahl $d$ kann aus den Primfaktoren $p$ und $q$ von $n$ bestimmt werden. Wenn also die Primfaktorzerlegung von $n$ gelingt, kann der private Schlüssel bestimmt werden. Deshalb ist es von grosser Wichtigkeit, die beiden Primzahlen $p$ und $q$ so gross zu wählen, dass eine Primfaktorzerlegung von $n$ nicht in nützlicher Zeit erfolgen kann.






[*] Rivest, Shamir, and Adleman. A method for obtaining digital signatures and public-key cryptosystems. Communications of the ACM, 21:120–126, 1978.


<a name="schluessel"></a>
### 2. Berechnung der Schlüssel

Das Vorgehen wird im Folgenden Schritt für Schritt erklärt. Wir wählen dazu zunächst die beiden Primzahlen, z. B. $p=47$ und $q=59$ und bilden daraus das Produkt $n=p*q$. 

In [1]:
p=47
q=59
n=p*q
print('Das Produkt lautet n =', n)

Das Produkt lautet n = 2773


Nun berechnen wir die Euler'sch $\phi$-Funktion: $\phi=(p-1)(q-1)$ Näheres dazu finden Sie in diesem [Wikipedia Artikel](https://de.wikipedia.org/wiki/Eulersche_Phi-Funktion).

In [2]:
phi=(p-1)*(q-1)
print('Der Eulersche Phi-Funktionswert lautet phi = ', phi)

Der Eulersche Phi-Funktionswert lautet phi =  2668


Dann wählen wir eine Zahl $e$ (z. B. eine Primzahl), die in jedem Fall teilerfremd zu $\phi(n)$ sein muss, also weder ein Teiler von $46$ noch von $58$ ist. Wir wählen $e=17$.

Dann gilt für den **g**rössten **g**emeinsamen **T**eiler mit Sicherheit: $ggT(\phi, e)=ggT(2668, 17)=1$

Damit lösen wir die Gleichung:

$$d\cdot e\ mod\ \phi = 1 \Rightarrow d\cdot e\ mod\ (p-1)(q-1) = d\cdot e\ mod\ 2668 = 1$$

Mit der Definition 

$$a\ mod\ b=a - \left\lfloor \frac{a}{b}\right\rfloor\cdot b$$

Dabei bedeutet $\lfloor \cdot \rfloor$ die **Gaussklammer**. Nach unten geschlossen, ordnet sie einer Zahl ihre nächstliegende, nicht grössere ganze Zahl zu, das heisst, dass auf die vorherige ganze Zahl abgerundet wird. Siehe dazu auch diesen [Artikel im Web](https://www.mathespass.at/klasse5/gaussklammer.php). 

In diesem Fall wird auf $1$ abgerundet. Deshalb ist die obige Bedingung gleichbedeutend mit

$$d\cdot e\ mod\ \phi = d\cdot e\ mod\ (p-1)(q-1) = d\cdot e - (p-1)(q-1)= 1$$

Was wir nach der gesuchten Zahl $d$ auflösen können:

$$d=\frac{1+(q-1)\cdot (p-1)}{e}=\frac{1+2668}{17}=157$$

Das lassen wir uns auch noch vom Computer ausrechnen:



In [3]:
e=17 # scheint nur mit 17 zu funktionieren!?
d=(1+(q-1)*(p-1))/e
d=int(d) # Umwandlung in Integer notwendig, sonst gibt es Rundungsfehler
print('Die Lösung von d*e mod phi = 1 ist: d= ', d)

Die Lösung von d*e mod phi = 1 ist: d=  157


Damit haben wir einen öffentlichen Schlüssel $(e, n)=(17, 2773)$ und einen privaten Schlüssel $(d, n)=(157, 2773)$ erzeugt.

<a name="verschl"></a>
### 3. Verschlüsseln einer Nachricht

Nun verschlüsseln wir mit dem privaten Schlüssel eine Botschaft (Message) $m$, z. B. für $m=1220$.

Dazu berechnen wir:

$$ x=m^e\ mod\ n$$

In [4]:
m=1220
x=m**e%n
print('Die verschlüsselte Zahl ist x = ', x)

Die verschlüsselte Zahl ist x =  1750


<a name="entschl"></a>
### 4. Entschlüsseln der Nachricht

Mit dem privaten (geheimen) Schlüssel $(d, n)=(157, 2773)$ rekonstruieren wir die Nachricht aus der verschlüsselten Zahl $x$:

$$mr=x^d\ mod\ n$$


In [5]:
mr=x**d%n
print('Die rekonstruierte Zahl ist: mr = ', mr)

Die rekonstruierte Zahl ist: mr =  1220


Zusammenfassend stellen wir noch einmal die wichtigsten Prameter zusammen:

In [6]:
print('e =', e, '   d =', d, '   n =', n)

e = 17    d = 157    n = 2773


#### Wichtig

Die Ver- und Entschlüsselung funktioniert nur bis $m<n$. In diesem Beispiel muss also $m<2773$ sein. Versuchen Sie es mit einer Zahl $m>2772$,  stimmt die rekonstruierte Zahl nicht mehr mit der eingegebenen Zahl überein. Probieren Sie es aus!

<a name="vareing"></a>
### 5. Programm mit variabler Eingabe

Wir wollen das Programm jetzt so gestalten, sodass wir zwei beliebige Primzahlen eingeben können und dass die Parameter $e$ und $d$ automatisch bestimmt werden.

Dazu definieren wir zuerst eine Python-Funktion, die mithilfe des **Euklid-Algorithmus** ([siehe hier](https://de.wikipedia.org/wiki/Euklidischer_Algorithmus)) den grössten gemeinsamen Teiler (**ggT**) zweier Zahlen berechnet:


In [4]:
# Funktion zur Bestimmung des ggT
def ggt(a, b):
    r=a%b
    while r:
        a=b
        b=r
        r=a%b
    return (b)

Nun folgt der Programmteil zur **Eingabe** der beiden Primzahlen:

In [5]:
# Hier erfolgt die Eingabe der beiden Primzahlen und die Berechnung von Phi
eingabe = input("Geben Sie zwei Primzahlen ein (durch Komma getrennt): ")
eingabe_string = eingabe.split(',')
p = int(eingabe_string[0])
q = int(eingabe_string[1])
n=p*q
phi=(p-1)*(q-1)

Geben Sie zwei Primzahlen ein (durch Komma getrennt):  67, 193


Im folgenden Programmteil werden die beiden Zahlen $e$ und $d$ berechnet. Dies geschieht rekursiv: Wir probieren einfach alle Zahlen (bis zum Maximum von $p-1$ und $q-1$) durch. Dabei stellen wir fest, dass es **viele Lösungen** gibt, die wir alle auflisten:

In [6]:
# Nun müssen Zahlen e gefunden werden, die teilerfremd zu p-1 und q-1 sind sowie die dazugehörigen d-Werte
bis=max([p-1, q-1])
print (bis)
for x in range (1, bis):
    if (ggt(p-1, x)==1) and (ggt(q-1, x)==1):
        e=x
        for d in range (1, (p-1)*(q-1)):
            if ((d*e)%((p-1)*(q-1))==1):
                print ('e = ', e, '    d = ', d)  

192
e =  1     d =  1
e =  5     d =  5069
e =  7     d =  5431
e =  13     d =  8773
e =  17     d =  8945
e =  19     d =  667
e =  23     d =  551
e =  25     d =  8617
e =  29     d =  437
e =  31     d =  3679
e =  35     d =  6155
e =  37     d =  685
e =  41     d =  8345
e =  43     d =  2947
e =  47     d =  9167
e =  49     d =  8017
e =  53     d =  5021
e =  59     d =  10739
e =  61     d =  8725
e =  65     d =  4289
e =  67     d =  9835
e =  71     d =  8567
e =  73     d =  9721
e =  79     d =  5935
e =  83     d =  6107
e =  85     d =  1789
e =  89     d =  4841
e =  91     d =  12115
e =  95     d =  10271
e =  97     d =  7969
e =  101     d =  7277
e =  103     d =  4183
e =  107     d =  11843
e =  109     d =  4069
e =  113     d =  785
e =  115     d =  5179
e =  119     d =  8519
e =  125     d =  11861
e =  127     d =  5887
e =  131     d =  1451
e =  133     d =  10957
e =  137     d =  185
e =  139     d =  547
e =  145     d =  10225
e =  149     d =  72

#### Verschlüsselung
Da es eine Vielzahl von Möglichkeiten gibt, müssen Sie zwei zusammengehörige Werte von $e$ und $d$ aus der Liste oben auswählen und in den Programmcode unten eintragen. Ebenso müssen Sie die zu verschlüsselnde Zahl $m$ im Listing unten eingeben. Dabei muss $m<n$ sein, sonst funktioniert der Algorithmus nicht.

In [7]:
#Wir wählen zum Beispiel:
e=137
d=185
print ('n = ', n)
m=12930
x=pow(m, e)%n
print('Die verschlüsselte Zahl ist x = ', x)

n =  12931
Die verschlüsselte Zahl ist x =  12930


#### Entschlüsselung

In [8]:
mr=pow(x, d)%n
print('Die rekonstruierte Zahl ist: mr = ', mr)

Die rekonstruierte Zahl ist: mr =  12930


In [9]:
print('Der hier benutzte öffentliche Schlüssel ist (e, n) = ( ', e, ', ', n, ')')
print()
print('Der hier benutzte private Schlüssel ist (d, n) = ( ', d, ', ', n, ')')

Der hier benutzte öffentliche Schlüssel ist (e, n) = (  137 ,  12931 )

Der hier benutzte private Schlüssel ist (d, n) = (  185 ,  12931 )


<a name="knacken"></a>
### 6. Verschlüsselung knacken

Wenn wir den den Parameter $d$ des privaten Schlüssels finden wollen, müssen wir $n$ in seine Primfaktoren zerlegen. Auch das machen wir hier rekursiv. Wir probieren einfach alle Zahlen durch. Falls $n\ mod\ x=0$, haben wir eine teilende Primzahl gefunden. Ist $n$ das produkt zweier Primzahlen, findet man diese schnell, sofern diese nicht sehr gross sind: 


In [10]:
print(n)
for x in range(2, n):
    if n%x==0:
        print(x)
        for d in range (1, n):
            if ((d*e)%((p-1)*(q-1))==1):
                print (    'd = ', d)  
    

12931
67
d =  185
d =  12857
193
d =  185
d =  12857


<a name="primzahlen"></a>
### 7. Die 168 Primzahlen zwischen 1 und 1000

Eine Primzahl ist eine natürliche Zahl **grösser** als 1, die nur durch 1 und sich selbst teilbar ist. Da 1 nur einen einzigen Teiler hat (nämlich sich selbst), erfüllt sie diese Bedingung nicht und gehört deshalb nicht zu den Primzahlen.

<img src='Bilder/Primzahlen_1000.PNG'>

<a name="experimente"></a>
### 8. Eigene Experimente

Hier sind die Programmteile noch einmal zusammengefasst. Sie müssen nur zwei Primzahlen aus der obigen Liste auswählen und eingeben. Danach wird eine Liste mit möglichen Werten für $e$ und $d$ ausgedruckt. Aus dieser können Sie sich einen gültigen öffentlichen und privaten Schlüssel aussuchen. Danach können Sie die Schlüssel zum Verschlüsseln und zum Entschlüsseln selbst ausprobieren. Achten Sie jeweils darauf, dass die zu verschlüsselnde Zahl $m$ kleiner als das Produkt der eingegebenen Primzahlen ist, also $m<p \cdot q=n$.

In [23]:
# Hier erfolgt die Eingabe der beiden Primzahlen und die Berechnung von Phi
eingabe = input("Geben Sie zwei Primzahlen ein (durch Komma getrennt): ")
eingabe_string = eingabe.split(',')
p = int(eingabe_string[0])
q = int(eingabe_string[1])
n=p*q
phi=(p-1)*(q-1)

# Funktion zur Bestimmung des ggT
def ggt(a, b):
    r=a%b
    while r:
        a=b
        b=r
        r=a%b
    return (b)

# Nun müssen Zahlen e gefunden werden, die teilerfremd zu p-1 und q-1 sind sowie die dazugehörigen d-Werte
bis=max([p-1, q-1])
print (bis)
for x in range (1, bis):
    if (ggt(p-1, x)==1) and (ggt(q-1, x)==1):
        e=x
        for d in range (1, (p-1)*(q-1)):
            if ((d*e)%((p-1)*(q-1))==1):
                print ('e = ', e, '    d = ', d)  

Geben Sie zwei Primzahlen ein (durch Komma getrennt):  101, 211


210
e =  1     d =  1
e =  11     d =  19091
e =  13     d =  8077
e =  17     d =  12353
e =  19     d =  16579
e =  23     d =  20087
e =  29     d =  5069
e =  31     d =  12871
e =  37     d =  3973
e =  41     d =  2561
e =  43     d =  3907
e =  47     d =  9383
e =  53     d =  8717
e =  59     d =  5339
e =  61     d =  6541
e =  67     d =  9403
e =  71     d =  11831
e =  73     d =  20137
e =  79     d =  4519
e =  83     d =  20747
e =  89     d =  15809
e =  97     d =  433
e =  101     d =  7901
e =  103     d =  8767
e =  107     d =  8243
e =  109     d =  10789
e =  113     d =  8177
e =  121     d =  11281
e =  127     d =  13063
e =  131     d =  5771
e =  137     d =  1073
e =  139     d =  15259
e =  143     d =  16007
e =  149     d =  16349
e =  151     d =  13351
e =  157     d =  8293
e =  163     d =  20227
e =  167     d =  503
e =  169     d =  11929
e =  173     d =  17237
e =  179     d =  18419
e =  181     d =  5221
e =  187     d =  1123
e =  191     d 

In [24]:
# Hier erfolgt die Eingabe der beiden Primzahlen und die Berechnung von Phi
eingabe = input("Wählen Sie eine mögliche Kombination von e und d (durch Komma getrennt), aus obiger Liste aus: ")
eingabe_string = eingabe.split(',')
e = int(eingabe_string[0])
d = int(eingabe_string[1])


Wählen Sie eine mögliche Kombination von e und d (durch Komma getrennt), aus obiger Liste aus:  97, 433


In [25]:
print('Der öffentliche Schlüssel ist:  (',e,',',n,')')
print('Der private Schlüssel ist:  (',d,',',n,')')

Der öffentliche Schlüssel ist:  ( 97 , 21311 )
Der private Schlüssel ist:  ( 433 , 21311 )


In [30]:
# Hier erfolgt die Eingabe der zu verschlüsselnden Zahl:
eingabe = input("Geben Sie die zu verschlüsselnde Zahl ein: ")
eingabe_string = eingabe.split(',')
m = int(eingabe)
x=pow(m, e)%n
print('Die verschlüsselte Zahl ist x = ', x)

Geben Sie die zu verschlüsselnde Zahl ein:  15100


Die verschlüsselte Zahl ist x =  14451


In [31]:
mr=pow(x, d)%n
print('Die rekonstruierte Zahl ist: mr = ', mr)

Die rekonstruierte Zahl ist: mr =  15100


**Aufgabe:** Gehen Sie zu dem konkreten Beispiel im Buch aud S. 263. Probieren Sie verschiedene Schlüssel aus. Falls Sie lust am Programmieren haben, erstellen Sie ein Programm, das einen Text gemäss der vorgeschlagenen Codierung in Zahlen umwandelt und anschliessend mit **rsa** verschlüsselt und wieder entschlüsselt.